熵： 
符号$x_i$的信息定义为      
$l(x_i)=-log_2p(x_i)$
所有类别所有可能值包含信息的期望值（数学期望）有以下公式 :
$H=-\sum^n_{i=1}p(x_i)log_2p(x_i)$  
n为分类的数量，熵值越大，随机变量的不确定性就越大  



决策树-信贷问题  
https://cuijiahua.com/blog/2017/11/ml_2_decision_tree_1.html  
属性 标注如下：
- 年龄：0代表青年，1代表中年，2代表老年；
- 有工作：0代表否，1代表是；
- 有自己的房子：0代表否，1代表是；
- 信贷情况：0代表一般，1代表好，2代表非常好；
- 类别(是否给贷款)：no代表否，yes代表是。

In [1]:
from math import log 
"""
创建测试数据集

"""
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],         #数据集
            [0, 0, 0, 1, 'no'],
            [0, 1, 0, 1, 'yes'],
            [0, 1, 1, 0, 'yes'],
            [0, 0, 0, 0, 'no'],
            [1, 0, 0, 0, 'no'],
            [1, 0, 0, 1, 'no'],
            [1, 1, 1, 1, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [2, 0, 1, 2, 'yes'],
            [2, 0, 1, 1, 'yes'],
            [2, 1, 0, 1, 'yes'],
            [2, 1, 0, 2, 'yes'],
            [2, 0, 0, 0, 'no']]
    labels = ['不放贷', '放贷']             #分类属性
    return dataSet, labels                #返回数据集和分类属性   

In [4]:
"""
计算给定数据集的熵值
"""
def calcShannonEnt(dataSet):
    numEntries=len(dataSet)#数据集行数
    labelCounts={}#保存每个标签出现的次数的字典
    for featVec in dataSet:#对每个特征向量进行统计
        currentLabel=featVec[-1]#提取标签的信息
        if currentLabel not in labelCounts.keys():#如果标签没有放入统计次数的字典则加入
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    shannonEnt=0.0
    for key in labelCounts:#计算熵
        prob=float(labelCounts[key])/numEntries#选择该标签的 概论
        shannonEnt-=prob*log(prob,2)#套用公式
    return shannonEnt
        

In [6]:
if __name__=='__main__':
    dataSet,features=createDataSet()
    print(dataSet)
    print(calcShannonEnt(dataSet))#打印经验熵

[[0, 0, 0, 0, 'no'], [0, 0, 0, 1, 'no'], [0, 1, 0, 1, 'yes'], [0, 1, 1, 0, 'yes'], [0, 0, 0, 0, 'no'], [1, 0, 0, 0, 'no'], [1, 0, 0, 1, 'no'], [1, 1, 1, 1, 'yes'], [1, 0, 1, 2, 'yes'], [1, 0, 1, 2, 'yes'], [2, 0, 1, 2, 'yes'], [2, 0, 1, 1, 'yes'], [2, 1, 0, 1, 'yes'], [2, 1, 0, 2, 'yes'], [2, 0, 0, 0, 'no']]
0.9709505944546686


如何选择特征，需要查看信息增益。信息增益是针对特征而言的，信息增益越大,特征对最终分类结果影响就越大。我们应该选择对分类结果影响最大的特征作为分类特征。  
- 条件熵$H(Y|X)$表示已知随机变量X的条件下随机变量Y的不确定性  
$H(Y|X)=\sum^n_{i=1}p_iH(Y|X=x_i)$  
$p_i=P(X=x_i),i=1,2...,n$
- 特征A对训练数据集D的信息增益g(D,A),定义集合D的经验熵H(D)与特征A条件下D的经验熵H(D|A)之差，即  
$g(D,A)=H(D)-H(D|A)$  
一般地，熵H(D)与条件熵H(D|A)之差称为互信息(mutual information)。决策树学习中的信息增益等价于训练数据集中类与特征的互信息 
设特征A有n个不同的取值{a1,a2,···,an}，根据特征A的取值将D划分为n个子集{D1,D2，···,Dn}，|Di|为Di的样本个数。记子集Di中属于Ck的样本的集合为Dik，即Dik = Di ∩ Ck，|Dik|为Dik的样本个数。于是经验条件熵的公式可以些为： 
$H(D|A)=-\sum ^n_{i=1}\frac{|D_i|}{|D|}H(D_i)$
$=-\sum ^n_{i=1} \frac{|D_i|}{|D|}\sum^k_{i=1}\frac{|D_{ik}|}{|D_i|}log_2\frac{|D_{ik}|}{|D_i|}$

In [1]:
"""
按照给定 特征划分数据集
dataSet--待划分的数据集
axis-- 划分数据集的特征
value--需要返回的特征的值
"""
def splitDataSet(dataSet,axis,value):
    retDataSet=[]#创建返回的数据集列表
    for featVec in dataSet:
        if featVec[axis]==value:
            reducedFeatVec=featVec[:axis]#去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])#将符合条件的添加到返回的数据集中
            retDataSet.append(reducedFeatVec)
    return retDataSet#返回划分后的数据集


In [2]:
"""
选择最优特征
dataSet--数据集
bestFeature--信息增益最大的特征的索引（返回

"""
def chooseBestFeatureToSplit(dataSet):
    numFeatures=len(dataSet[0])-1
    baseEntropy=calcShannonEnt(dataSet)#计算数据集的香农熵
    bestInfoGain=0.0#信息增益
    bestFeature=-1#最优特征的索引值
    for i in range(numFeatures):#遍历所有特征
        #获取dataSet第i个所有特征
        featList=[example[i] for example in dataSet]
        uniqueVals=set(featList)#创建set,元素不可重复
        newEntropy=0.0#经验条件熵
        #计算信息增益
        for value in uniqueVals:
            subDataSet=splitDataSet(dataSet,i,value)#subDataSet划分后的子集
            prob=len(subDataSet)/float(len(dataSet))#计算子集的概率
            newEntropy+=prob*calcShannonEnt(subDataSet)#计算经验条件熵
        infoGain=baseEntropy-newEntropy #信息增益
        print("第%d个特征的增益为%.3f" % (i, infoGain))            #打印每个特征的信息增益
        if (infoGain > bestInfoGain):                             #计算信息增益
            bestInfoGain = infoGain                             #更新信息增益，找到最大的信息增益
            bestFeature = i                                     #记录信息增益最大的特征的索引值
    return bestFeature                                             #返回信息增益最大的特征的索引值
 

In [4]:
from math import log 
"""
创建测试数据集

"""
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'no'],         #数据集
            [0, 0, 0, 1, 'no'],
            [0, 1, 0, 1, 'yes'],
            [0, 1, 1, 0, 'yes'],
            [0, 0, 0, 0, 'no'],
            [1, 0, 0, 0, 'no'],
            [1, 0, 0, 1, 'no'],
            [1, 1, 1, 1, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [1, 0, 1, 2, 'yes'],
            [2, 0, 1, 2, 'yes'],
            [2, 0, 1, 1, 'yes'],
            [2, 1, 0, 1, 'yes'],
            [2, 1, 0, 2, 'yes'],
            [2, 0, 0, 0, 'no']]
    labels = ['不放贷', '放贷']             #分类属性
    return dataSet, labels                #返回数据集和分类属性   

In [6]:
"""
计算给定数据集的熵值
"""
def calcShannonEnt(dataSet):
    numEntries=len(dataSet)#数据集行数
    labelCounts={}#保存每个标签出现的次数的字典
    for featVec in dataSet:#对每个特征向量进行统计
        currentLabel=featVec[-1]#提取标签的信息
        if currentLabel not in labelCounts.keys():#如果标签没有放入统计次数的字典则加入
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    shannonEnt=0.0
    for key in labelCounts:#计算熵
        prob=float(labelCounts[key])/numEntries#选择该标签的 概论
        shannonEnt-=prob*log(prob,2)#套用公式
    return shannonEnt
        

In [7]:
if __name__ == '__main__':
    dataSet, features = createDataSet()
    print("最优特征索引值:" + str(chooseBestFeatureToSplit(dataSet)))

第0个特征的增益为0.083
第1个特征的增益为0.324
第2个特征的增益为0.420
第3个特征的增益为0.363
最优特征索引值:2


以上得 
最优特征索引值为2 也就是特征A3(房子） 
